In [47]:
url_1 = "https://odre.opendatasoft.com/explore/embed/dataset/conso-departement-annuelle/table/?disjunctive.libelle_departement&disjunctive.libelle_region&disjunctive.e_operateurs&disjunctive.g_operateurs&refine.annee=2021&dataChart=eyJxdWVyaWVzIjpbeyJjaGFydHMiOlt7InR5cGUiOiJjb2x1bW4iLCJmdW5jIjoiQVZHIiwieUF4aXMiOiJjb25zb3RvdGFsZSIsInNjaWVudGlmaWNEaXNwbGF5Ijp0cnVlLCJjb2xvciI6IiM2NmMyYTUifV0sInhBeGlzIjoibGliZWxsZV9kZXBhcnRlbWVudCIsIm1heHBvaW50cyI6NTAsInNvcnQiOiIiLCJjb25maWciOnsiZGF0YXNldCI6ImNvbnNvLWRlcGFydGVtZW50LWFubnVlbGxlIiwib3B0aW9ucyI6eyJkaXNqdW5jdGl2ZS5saWJlbGxlX2RlcGFydGVtZW50Ijp0cnVlLCJkaXNqdW5jdGl2ZS5saWJlbGxlX3JlZ2lvbiI6dHJ1ZSwiZGlzanVuY3RpdmUuZV9vcGVyYXRldXJzIjp0cnVlLCJkaXNqdW5jdGl2ZS5nX29wZXJhdGV1cnMiOnRydWUsInJlZmluZS5hbm5lZSI6IjIwMjEifX19XSwidGltZXNjYWxlIjoiIiwiZGlzcGxheUxlZ2VuZCI6dHJ1ZSwiYWxpZ25Nb250aCI6dHJ1ZX0%3D&location=3,17.56025,53.4375&basemap=jawg.light"
url_2 = "https://www.insee.fr/fr/statistiques/6436484?sommaire=6036904#tableau-figure1_radio1"
url_3 = "https://odre.opendatasoft.com/explore/dataset/temperature-quotidienne-departementale/information/?disjunctive.departement"
url_4 = "https://www.insee.fr/fr/statistiques/6436484?sommaire=6036904#tableau-figure1_radio1"
url_5 = "https://www.observatoire-des-territoires.gouv.fr/outils/cartographie-interactive/#c=indicator&i=insee_rp_hist_1968.part_logt_vacant&s=2020&view=map9"
url_6 = "https://ufe-electricite.fr/watt-the-carte/deploiement-bornes-de-recharge-en-france/dans-les-territoires/"
url_7 = "https://www.carburants.org/borne-electrique/departements/"

In [2]:
# Installation des packages

!pip install -q lxml
!pip install webdriver-manager
!pip install BeautifulSoup4

import requests
import re
import bs4
import lxml
import pandas as pd
import urllib

from urllib import request

In [ ]:
# API

In [ ]:
# On récupere les données de la conso totale d'energie par département, sur le site opendatasoft

In [15]:
# Création de l'url de l'API

root_api_1 = "https://odre.opendatasoft.com"
url_api_1 = root_api_1+"/api/explore/v2.1/catalog/datasets/conso-departement-annuelle/records?select=code_departement%2C%20libelle_departement%2C%20consototale&where=annee%20%3D%20date%272021%27&order_by=code_departement&limit=99&exclude=libelle_departement%3A971%2C972%2C973%2C974&refine=annee%3A%222021%22"

In [17]:
# Récupération des données

req = requests.get(url_api_1)
conso_energie = req.json()

# Extraire la liste de résultats du dictionnaire
results = conso_energie.get('results', [])
df = pd.DataFrame(results)

# On choisit les variables
df = df[['code_departement', 'libelle_departement', 'consototale']]

# Affichage de la table
df.head()

,code_departement,libelle_departement,consototale
0,01,Ain,9.921409e+06
1,02,Aisne,8.628880e+06
2,2A,Corse-du-Sud,1.008356e+06
3,2B,Haute-Corse,1.003554e+06
4,03,Allier,6.912756e+06


In [ ]:
# On récupere les données de température par département, sur le site opendatasoft

In [ ]:
# Notre jeu de donnée nous donne la température quotidienne dans chaque département. 
# Ne prendre qu'un seul relevé de température augmente énormément la variance de la prédiction si on introduit la température dans le modèle
# On propose donc de récupérer les relevés de températures moyen à chaque 1er du mois, et d'en faire une moyenne 

In [5]:
# On test pour la table des températures du mois de janvier

root_api = "https://odre.opendatasoft.com"
url_api_jan = root_api+"/api/explore/v2.1/catalog/datasets/temperature-quotidienne-departementale/records?select=date_obs%2Ccode_insee_departement%2Cdepartement%2Ctmoy&where=date_obs%3Ddate'2021-01-01'&order_by=code_insee_departement&limit=99&refine=date_obs%3A%222021%22"

req = requests.get(url_api_jan)
temp_jan = req.json()
results = temp_jan.get('results', [])
df = pd.DataFrame(results)
df = df[['date_obs', 'code_insee_departement', 'departement','tmoy']]
df.head()

,date_obs,code_insee_departement,departement,tmoy
0,2021-01-01,01,Ain,1.50
1,2021-01-01,02,Aisne,2.15
2,2021-01-01,2A,Corse-du-Sud,8.71
3,2021-01-01,2B,Haute-Corse,9.12
4,2021-01-01,03,Allier,2.45


In [13]:
# On essaye d'automatiser le processus

liste_url=[]
for i in range(1, 13):
    date_str = f"2021-{i:02d}-01"  # Utilisation de :02d pour formater i avec deux chiffres
    url_api = f"{root_api}/api/explore/v2.1/catalog/datasets/temperature-quotidienne-departementale/records?select=date_obs%2Ccode_insee_departement%2Cdepartement%2Ctmoy&where=date_obs%3Ddate'{date_str}'&order_by=code_insee_departement&limit=99&refine=date_obs%3A%222021%22"
    liste_url.append(url_api)

liste_tab=[]
for url_api in liste_url:
    req = requests.get(url_api)
    temp = req.json()
    results = temp.get('results', [])
    df = pd.DataFrame(results)
    df = df[['date_obs', 'code_insee_departement', 'departement','tmoy']]
    liste_tab.append(df)

# Il reste à fusionner tous les tableaux, et calculer la moyenne des températures moyennes quotidiennes

In [54]:
# Webscrapping

In [ ]:
# Code département, département et population

In [18]:
# Nous avons besoin de scraper la donnée "population" sur chaque page Wikipédia des départements, et pour cela, nous devons obtenir les URL de chaque page. 
# Ainsi, nous créons une liste qui recense tous les URL nécessaires pour accéder à chaque page Wikipédia et extraire la donnée de population.

url_dep = "https://fr.wikipedia.org/wiki/D%C3%A9partement_fran%C3%A7ais"
text_dep = request.urlopen(url_dep).read()
page_dep = bs4.BeautifulSoup(text_dep, 'lxml')
tableau_dep = page_dep.find('div', {'class':'colonnes'})
lignes_dep = tableau_dep.find_all('li')

list_url=[]
for i in range(len(lignes_dep)):
    if re.search(r'href="([^"]*)"', str(lignes_dep[i])).group(1) not in ["#cite_note-collectivitésOM-178",'/wiki/Guadeloupe','/wiki/La_R%C3%A9union'] :
        num_dep = lignes_dep[i].find('code').text.strip()
        list_url.append(["https://fr.wikipedia.org"+re.search(r'href="([^"]*)"', str(lignes_dep[i])).group(1),num_dep])

In [19]:
# Nous automatisons le webscraping d'une page Wikipédia, et chaque résultat est stocké dans un dictionnaire.

dico_dep = dict()
k=1
for url in list_url:
    text = request.urlopen(url[0]).read()
    page = bs4.BeautifulSoup(text, 'lxml')
    tableau = page.find('table',{'class':'infobox_v2 noarchive'})
    tableau_body = tableau.find('tbody')
    lignes = tableau_body.find_all('tr')

    # On webscrapp le nom du département
    
    nom_dep_html = lignes[0].find('td')
    nom_dep_elements = [item for item in nom_dep_html.contents if item.name != 'small'] # On retire les balises <small> qui correpondent aux sous-titres des départements (parfois traduction en breton)
    nom_dep_nouveau_html = bs4.BeautifulSoup('', 'lxml')
    nom_dep_nouveau_html.extend(nom_dep_elements)
    nom_dep_texte = nom_dep_nouveau_html.text.strip()
    nom_dep = re.sub(r'\([^)]*\)', '', nom_dep_texte)

    # On définie le numéro du département
    
    num_dep = url[1]

    # On webscrapp la population du département
    
    for ligne in lignes:
        cols1 = ligne.find_all('th')
        cols2 = ligne.find_all('td')
        cols = [ele.text.strip() for ele in cols1]+[elem.text.strip() for elem in cols2]
        if cols[0]=='Population' or cols[0]=='Populationmunicipale':
            population = re.sub(r'hab.*', '', cols[1])
    dico_dep[num_dep] = {'nom_dep': nom_dep, 'population': population}


In [20]:
# On transforme le dictionnaire en base de données

data_dep = pd.DataFrame.from_dict(dico_dep,orient='index').reset_index()
data_dep = data_dep.rename(columns={'index' : 'Numéro','nom_dep':'Nom','population':'Population'}) 

In [22]:
# On visualise la base de donnée

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
data_dep.head()

,Numéro,Nom,Population
0,01,Ain,657 856
1,02,Aisne,529 374
2,03,Allier,335 628
3,04,Alpes-de-Haute-Provence,165 451
4,05,Hautes-Alpes,140 605


In [249]:
# Test pour récupérer la population sur une page wikipédia

# On récupère l'url de la page wikipédia et on importe le texte HTML de la page

url_Loire_Atlantique = "https://fr.wikipedia.org/wiki/Loire-Atlantique"
text_Loire_Atlantique = request.urlopen(url_Loire_Atlantique).read()

In [250]:
# On recherche de manière efficace les balises de la page
population_Loire_Atlantique = 0
page_Loire_Atlantique = bs4.BeautifulSoup(text_Loire_Atlantique, 'lxml')

# On récupère le tableau souhaité
tableau_Loire_Atlantique = page_Loire_Atlantique.find('table',{'class':'infobox_v2 noarchive'})

# On extrait le corps du tableau
tableau_Loire_Atlantique_body = tableau_Loire_Atlantique.find('tbody')

# On extrait toutes les lignes
lignes = tableau_Loire_Atlantique_body.find_all('tr')


#On affiche uniquement le texte et on supprime le code (avec la fonction .text.strip) et on extrait la ligne population du tableau
for ligne in lignes:
    cols1 = ligne.find_all('th')
    cols2 = ligne.find_all('td')
    cols = [ele.text.strip() for ele in cols1]+[elem.text.strip() for elem in cols2]
    if cols[0]=='Population':
        population_Loire_Atlantique = cols[1]

print(population_Loire_Atlantique)

1 445 171 hab. (2020)


In [ ]:
# Webscrapping du niveau de vie

In [28]:
text = request.urlopen("https://www.insee.fr/fr/statistiques/6436484?sommaire=6036904#tableau-figure1_radio1").read()
page = bs4.BeautifulSoup(text, 'lxml')
tableau = page.find('table',{'class':'tableau-produit'})
tableau_body = tableau.find('tbody')
lignes = tableau_body.find_all('tr')

In [41]:
dico_niveau_vie={}
for ligne in lignes:
    nom_dep = ligne.find('th').text.strip()
    niveau_vie_dep = ligne.find('td').text.strip()
    dico_niveau_vie[nom_dep] = {'Niveau de Vie': niveau_vie_dep}

tableau_niveau_vie = pd.DataFrame.from_dict(dico_niveau_vie,orient='index').reset_index()
tableau_niveau_vie = tableau_niveau_vie.rename(columns={'index':'Nom'}) 
tableau_niveau_vie

,Nom,Niveau de Vie
0,Ain,23 490
1,Aisne,19 880
2,Allier,20 570
3,Alpes-Maritimes,22 300
4,Alpes-de-Haute-Provence,20 690
5,Ardennes,19 840
6,Ardèche,21 010
7,Ariège,20 010
8,Aube,20 580
9,Aude,19 550


In [55]:
# Webscrapping des logements vacants

text_logements_vacants = request.urlopen(url_5).read()
page_logements_vacants = bs4.BeautifulSoup(text_logements_vacants, 'lxml')
tableau_logements_vacants = page_logements_vacants.find('table', {'id': 'tm_datatable'})
print(tableau_logements_vacants)
## A partir d'ici, pas sûr d'avoir récuperer la bonne partie du code HTML qui code le tableau



None


In [51]:
print(page_logements_vacants)

In [53]:
text_logements_vacants = request.urlopen(url_5).read()
page_logements_vacants = bs4.BeautifulSoup(text_logements_vacants, 'lxml')
print(page_logements_vacants)